# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## 1 Подготовка

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.stem import WordNetLemmatizer 
import re 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("https://code.s3.yandex.net/datasets/toxic_comments.csv")

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.describe()

,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [6]:
X = df["text"].values.astype('U')

In [7]:
y = df["toxic"]

In [8]:
#функция для лемматизации
def lemma(text):
    l = WordNetLemmatizer()
    return ["".join(l.lemmatize(w)) for w in text] 

In [9]:
#функция для очистки комментариев от ненужных символов
def clear_text(text):
    return [" ".join((re.sub('[^a-zA-Z]', ' ', w)).split()) for w in text]

In [10]:
X = lemma(clear_text(X))

In [11]:
X_train, X_test, y_train, y_test =\
train_test_split(X, y, test_size=0.2, random_state=47)

In [12]:
stopwords = set(nltk_stopwords.words('russian'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords, min_df = 0.0001) 

In [13]:
count_tf_idf.fit(X_train)

TfidfVectorizer(min_df=0.0001,
                stop_words={'а', 'без', 'более', 'больше', 'будет', 'будто',
                            'бы', 'был', 'была', 'были', 'было', 'быть', 'в',
                            'вам', 'вас', 'вдруг', 'ведь', 'во', 'вот',
                            'впрочем', 'все', 'всегда', 'всего', 'всех', 'всю',
                            'вы', 'где', 'да', 'даже', 'два', ...})

In [16]:
X_train = count_tf_idf.transform(X_train)

In [17]:
X_test = count_tf_idf.transform(X_test)

In [18]:
#функция, которая применяет методы fit, predict и считает и выводит F score
def fit_predict_evaluate(model, X_train=X_train, y_train=y_train, 
                         y_test=y_test, X_test=X_test):
    model.fit(X_train, y_train)
    return print("F score: ", f1_score(y_test, model.predict(X_test)))

In [19]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(127656, 16500) (31915, 16500) (127656,) (31915,)


## 2 Обучение

Посмотрим, что лучше работает "из коробки" и подберем параметры для лучшей модели.

### 2.1 LogisticRegression

In [20]:
model = LogisticRegression()

In [21]:
fit_predict_evaluate(model)

F score:  0.7523270669830261


### 2.2 DecisionTree

In [22]:
model = DecisionTreeClassifier()

In [23]:
fit_predict_evaluate(model)

F score:  0.6840370428504159


### 2.3 RandomForest

In [24]:
model = RandomForestClassifier()

In [25]:
fit_predict_evaluate(model)

F score:  0.6907095256762056


### Подбор параметров

Лучше всего работает логистическая регрессия. Результат 0.752 - немного выше нужного результата. Попробуем его еще улучшить с помощью перебора параметров по сетке.

In [26]:
grid={"C":np.logspace(-3,3,3), "penalty":["l1","l2"], 
      "class_weight": ['balanced', None]}

In [27]:
model = LogisticRegression()

In [28]:
clf = GridSearchCV(estimator=model, param_grid=grid,  cv=3, scoring="f1")
fit_predict_evaluate(clf)

F score:  0.7667731629392973


Подбор параметров улучшил результат до 0.7667. Посмотрим, поможет ли изменение параметра solver.

In [29]:
clf.best_params_

{'C': 1000.0, 'class_weight': None, 'penalty': 'l2'}

In [30]:
for i in ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]:
    model = LogisticRegression(C=1000.0, class_weight=None, 
                               penalty= 'l2', solver=i)
    model.fit(X_train, y_train)
    print(i, f1_score(y_test, model.predict(X_test)))

newton-cg 0.7217284707324548
lbfgs 0.7667731629392973
liblinear 0.725622885266072
sag 0.7476489028213166
saga 0.7609352632416096


Изменение solver не помогло. Самый лучший вариант - solver по умолчанию lbfgs. Но он выдает ошибку о том, что минимум функции заданным количеством операций не найдем. Попробуем изменить количество итераций.

In [42]:
model = LogisticRegression(C=1000.0, class_weight=None, penalty= 'l2', 
                           max_iter=50)

In [43]:
fit_predict_evaluate(model)

F score:  0.7972861161674665


In [33]:
precision_score(y_test, model.predict(X_test))

0.8625134264232008

In [34]:
recall_score(y_test, model.predict(X_test))

0.7412307692307692

## Выводы

Луший результат дала модель LogisticRegression с параметрами C=1000.0, class_weight=None, penalty= 'l2', max_iter=50. F score получился равным 0.7972. Если бы целевой метрикой был не F score, я бы дальше максимизировала recall, потому что в данной задаче нам выгоднее отловить все негативные комметарии, цена ошибки скорее всего не так велика. Поэтому в данном случае для нас важен recall.